In [1]:
import os
import io
import csv
import sys
import re
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math


In [2]:
def getData1D(filepath):
    file = open(filepath)
    reader = csv.reader(file)
    data_str = list(reader)
    file.close()
    X = [float(x[0]) for x in data_str]
    return X

def getData4D(filepath):
    file = open(filepath)
    reader = csv.reader(file)
    data_str = list(reader)
    file.close()
    X1 = [float(x[0]) for x in data_str]
    X2 = [float(x[1]) for x in data_str]
    X3 = [float(x[2]) for x in data_str]
    X4 = [float(x[3]) for x in data_str]
    return X1, X2, X3, X4

def getData5D(filepath):
    file = open(filepath)
    reader = csv.reader(file)
    data_str = list(reader)
    file.close()
    X1 = [float(x[0]) for x in data_str[1:]]
    X2 = [float(x[1]) for x in data_str[1:]]
    X3 = [float(x[2]) for x in data_str[1:]]
    X4 = [float(x[3]) for x in data_str[1:]]
    rob = [float(x[4]) for x in data_str[1:]]
    return X1, X2, X3, X4, rob

# Parameters (set it up YO-self)

In [26]:
scenario_names = ['LVM1', 'LVM2', 'LVDAD', 'LVD', 'POV_UnprotectedLeft']
scenario = scenario_names[3]
# print(scenario)
#
res_path = "../../results/"+scenario
dir_list = next(os.walk(res_path))[1]
print (dir_list)
#
scenario_folders = [res_path+'/'+f for f in dir_list] # Select scenario folders
print('folders to analyze ', scenario_folders)
#
robustness_fn = 'f_history.csv'
params_fn = 'x_history.csv'
#
X1_name = 'alpha_lon_accel_max'
X2_name = 'alpha_lon_brake_max'
X3_name = 'alpha_lon_brake_min'
X4_name = 'response_time'

['LVD1']
folders to analyze  ['../../results/LVD/LVD1']


# Creating a dictionary to find duplicated data points


In [27]:
## creating a dictionary to find duplicated data points

X1 = []
X2 = []
X3 = []
X4 = []
robustness = []
data_dict = defaultdict(list)

for folder in scenario_folders:
    print("reading folder ", folder)
    rob_fn = os.path.join(folder, robustness_fn)
    par_fn = os.path.join(folder, params_fn)
    
    robustness = getData1D(rob_fn)
    X1,X2,X3,X4 = getData4D(par_fn)
    print("data size ", len(robustness))
    
    for i in range(len(robustness)):
        str_keys = [str(X1[i]), str(X2[i]), str(X3[i]), str(X4[i])]
        key = '_'.join(str_keys)

        if data_dict[key]:
            data_dict[key][X1_name].append(X1[i])
            data_dict[key][X2_name].append(X2[i])
            data_dict[key][X3_name].append(X3[i])
            data_dict[key][X4_name].append(X4[i])
            data_dict[key]['robustness'].append(robustness[i])
            print('repeated params:', key, ', robustness:', data_dict[key]['robustness'])

        else: 
            data_dict[key] = {X1_name:[], X2_name:[], X3_name:[], X4_name:[], 'robustness':[]}
            data_dict[key][X1_name].append(X1[i])
            data_dict[key][X2_name].append(X2[i])
            data_dict[key][X3_name].append(X3[i])
            data_dict[key][X4_name].append(X4[i])
            data_dict[key]['robustness'].append(robustness[i])
        
    print("final data size ", len(data_dict.keys()))
  

reading folder  ../../results/LVD/LVD1
data size  3001
final data size  3001


# Save data to csv file 

In [28]:
### Save data to csv file 
with open(os.path.join(res_path, scenario + '_combined.csv'), 'w') as f:
    fieldnames = [X1_name, X2_name, X3_name, X4_name, 'robustness']
    writer = csv.writer(f)
    writer.writerow(fieldnames)
    
    for entry in data_dict:
        vals = [data_dict[entry][X1_name][0], 
                data_dict[entry][X2_name][0], 
                data_dict[entry][X3_name][0], 
                data_dict[entry][X4_name][0], 
                data_dict[entry]['robustness'][0]]
        writer.writerow(vals)     

# Read data from a csv to pandas data frame

In [29]:
# Read data from a csv to pandas data frame
data = pd.read_csv(os.path.join(res_path, scenario + '_combined.csv'), index_col=False)

# sort values by robustness
data = data.sort_values(by=['robustness'])

rob_min = data['robustness'].iloc[0]
rob_max = data['robustness'].iloc[-1]

# create bins for data binning
bins = range(0, math.ceil(rob_max)+1, 1)
# print(bins)

In [30]:
## separate data into bins and extract min max
min_bin_params = []
max_bin_params = []

min_max_vals = []
for i in bins:
#     print(i)
    if i == 0:
        beg = i
        end = i
        data_aux = data[data['robustness'] <= end]
    else:
        beg = i-1
        end = i
        data_aux = data[data['robustness'] <= end]
        data_aux = data_aux[data_aux['robustness'] > beg]

    if len(data_aux) == 0:
        
        min_bin_params.append([-1])
        max_bin_params.append([-1])
        
        min_max_vals.append([-1,-1,-1,-1,-1,-1,-1,-1, i])

    else:  
        
        min_bin_params.append(list(data_aux.iloc[0]))
        max_bin_params.append(list(data_aux.iloc[-1]))

        min_max_vals.append([data_aux['alpha_lon_accel_max'].min(),
               data_aux['alpha_lon_accel_max'].max(),
               data_aux['alpha_lon_brake_max'].min(),
               data_aux['alpha_lon_brake_max'].max(),
               data_aux['alpha_lon_brake_min'].min(),
               data_aux['alpha_lon_brake_min'].max(),
               data_aux['response_time'].min(),
               data_aux['response_time'].max(), 
                   i])

        
### Save results to csv        
with open(os.path.join(res_path, scenario + '_param_bounds.csv'), 'w') as f:
    
    fieldnames = [X1_name+'_min', X1_name+'_max', X2_name+'_min', X2_name+'_max',
                  X3_name+'_min', X3_name+'_max', X4_name+'_min', X4_name+'_max',
                  'robustness_bin']
    writer = csv.writer(f)
    writer.writerow(fieldnames)
    
    for row in min_max_vals:
        writer.writerow(row) 

    
    fieldnames = [X1_name, X2_name,
                  X3_name, X4_name,
                  'robustness_min', 'bin']
    writer = csv.writer(f)
    writer.writerow(fieldnames)
    for i, row in enumerate(min_bin_params):
        writer.writerow(row+[i]) 
    
    fieldnames = [X1_name, X2_name,
                  X3_name, X4_name,
                  'robustness_max', 'bin']
    writer = csv.writer(f)
    writer.writerow(fieldnames)
        
    for i, row in enumerate(max_bin_params):
        writer.writerow(row+[i]) 